In [113]:
%load_ext autoreload
%autoreload

import io
import os
import sys
import django
import glob
import json
import re
import requests
from IPython.core.debugger import set_trace
from pathlib import Path
from django.core.files.base import ContentFile
from rest_framework import serializers
from tqdm.notebook import tqdm
from google.cloud import vision
from google.cloud import storage
from PIL import Image, ImageSequence
from gensim.models import Doc2Vec
from IPython.core.debugger import set_trace
import IPython
import collections
from googletrans import Translator
from google.cloud import translate_v2 as translate

parentdir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parentdir)
django.setup()

import getch.models as m
from django.db.models import F, Q, Sum, Count, Case, When, IntegerField, Value
from django.apps import apps
from django_currentuser.middleware import get_current_user
from datetime import datetime, timedelta
import time
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [114]:
# trans = m.Transaction.objects.select_related('sender','receiver'); trans
trans = m.Transaction.objects.select_related(
    'sender',
    'sender__boo','sender__boo__user',
    'sender__raffle','sender__raffle__item',
    'sender__shoptem','sender__shoptem__item',
    'sender__coffeecoupon','sender__coffeecoupon__item',
    'sender__support','sender__support__brand',
    'receiver',
    'receiver__boo','receiver__boo__user',
    'receiver__raffle','receiver__raffle__item',
    'receiver__support','receiver__support__brand',
    'receiver__shoptem','receiver__shoptem__item',
    'receiver__coffeecoupon','receiver__coffeecoupon__item',
); trans

<QuerySet [<Transaction: quantlab | quantlab@kakao.com > KINTO TUMBLER 350ML - 39000 | 2021-08-07 20:38:17>, <Transaction: quantlab | quantlab@kakao.com > POSTTO | 포스토 | 2021-08-06 16:21:02>, <Transaction: quantlab | quantlab@kakao.com > BADBOY | 배드보이 | 2021-08-08 14:43:50>, <Transaction: quantlab | quantlab@kakao.com > POSTTO | 포스토 | 2021-08-06 16:21:02>, <Transaction: quantlab | quantlab@kakao.com > BADBOY | 배드보이 | 2021-08-08 14:43:50>, <Transaction: quantlab | quantlab@kakao.com > POSTTO | 포스토 | 2021-08-06 16:21:02>, <Transaction: quantlab | quantlab@kakao.com > STARBUCKS LAMP BK - 0 | 2021-08-07 20:37:53>, <Transaction: quantlab | quantlab@kakao.com > TAW&TOE ZEROVITY™ Flip Flop, SIZE M - 38000 | 2021-08-07 23:34:16>, <Transaction: quantlab | quantlab@kakao.com > KINTO TUMBLER 350ML - 39000 | 2021-08-07 20:38:17>, <Transaction: quantlab | quantlab@kakao.com > JOHNBER | 존버 스포츠 | 2021-08-10 18:02:20>, <Transaction: quantlab | quantlab@kakao.com > BADBOY | 배드보이 | 2021-08-08 14:43:50>,

# 모든 트랜젝션 가져오기

In [115]:
trans_list = []
for _t in trans:
    tr = {}
    tr_type = _t.get_type_display()
    tr['user'] = str(_t.sender) if tr_type in ['래플','후원'] else str(_t.receiver)
    tr['when'] = str(_t.when.date())
    tr['type'] = tr_type
    tr['amount'] = -_t.amount if tr_type in ['래플','후원'] else _t.amount
    trans_list.append(tr)

# 트랜젝션 횟수 시계열

In [152]:
counts = pd.DataFrame(trans_list).groupby(['user','type','when']).count().unstack().fillna(0)
counts = counts.groupby('type').sum()[counts.columns[-23:-1]].drop(['첫방문환영','쇼핑','커피쿠폰','테스트 INFLOW']).astype('int64')
#counts.style.set_table_attributes('style="font-size: 10px"')
counts.T

type               관심스타일입력  관심아이템입력  기본정보입력  래플  리서치참여  매일밸런스게임  옷장넣기  출첵게임  \
       when                                                                   
amount 2021-09-13        0        0       3   0     32      402     0    30   
       2021-09-14        1        0       0   0      6      243     0    18   
       2021-09-15        0        0       2   3     23      499     0    19   
       2021-09-16        1        1       3   2     80      712     2    27   
       2021-09-17        0        0       2  12     18     1000     3    37   
       2021-09-18        1        1      10   7     58      639     4    21   
       2021-09-19        1        1       1  13     30      751     6    15   
       2021-09-20        0        0       0   0      8      819     1    16   
       2021-09-21        0        0       2   1     33      775     1    24   
       2021-09-22        0        0       3   2     19      886     3    25   
       2021-09-23        0        0       2   1     28     1041     4    28   
       2021-09-24        2        2       4   5     42      988     2    21   
       2021-09-25        0        0       2   4     37     1167     2    26   
       2021-09-26        2        2       4   4     41     1118     0    32   
       2021-09-27        2        1       4   2     36      853     1    20   
       2021-09-28        1        1       3   3     40      918     3    41   
       2021-09-29        1        1       3   1     42     1080     2    33   
       2021-09-30        2        2       7   3    119     2048     1    52   
       2021-10-01        1        1       6   9     70     1696     0    61   
       2021-10-02        0        0       1   5     16     1683     1    31   
       2021-10-03        1        1       0   2     23     1746     2    25   
       2021-10-04        0        0       0   1     19     1616     5    30   

type               후원  
       when            
amount 2021-09-13   0  
       2021-09-14   0  
       2021-09-15   5  
       2021-09-16   6  
       2021-09-17   3  
       2021-09-18   0  
       2021-09-19   0  
       2021-09-20   0  
       2021-09-21   0  
       2021-09-22   0  
       2021-09-23   0  
       2021-09-24   0  
       2021-09-25   1  
       2021-09-26   0  
       2021-09-27   0  
       2021-09-28   1  
       2021-09-29   1  
       2021-09-30   0  
       2021-10-01   0  
       2021-10-02   0  
       2021-10-03   0  
       2021-10-04   0

# 트랜젝션 금액 시계열

In [153]:
amounts = pd.DataFrame(trans_list).groupby(['user','type','when']).sum().unstack().fillna(0)
amounts = amounts.groupby('type').sum()[amounts.columns[-23:-1]].drop(['첫방문환영','쇼핑','커피쿠폰','테스트 INFLOW']).astype('int64')
#amounts.style.set_table_attributes('style="font-size: 10px"')
amounts.T

type               관심스타일입력  관심아이템입력  기본정보입력    래플  리서치참여  매일밸런스게임  옷장넣기  출첵게임  \
       when                                                                     
amount 2021-09-13        0        0     300     0   3640      400     0   240   
       2021-09-14       50        0       0     0    180      243     0   140   
       2021-09-15        0        0     200  -288   2020      475     0   130   
       2021-09-16       50       50     300  -143   8010      700     2   230   
       2021-09-17        0        0     200 -1050   2140      900     3   230   
       2021-09-18       50       50     400  -641   6270      635     4   190   
       2021-09-19       50       50     100 -1398   2770      685     6   110   
       2021-09-20        0        0       0     0    510      801     1   140   
       2021-09-21        0        0     200   -77   3670      735     1   180   
       2021-09-22        0        0     300  -222   1380      846     3   210   
       2021-09-23        0        0     200   -77   2980      973     4   200   
       2021-09-24      100      100     400  -464   4440      976     2   170   
       2021-09-25        0        0     200  -398   3770     1135     2   200   
       2021-09-26      100      100     400  -410   3890     1098     0   240   
       2021-09-27      100       50     400  -210   3170      831     1   200   
       2021-09-28       50       50     300  -288   4550      808     3   310   
       2021-09-29       50       50     300   -77   4790     1060     2   290   
       2021-09-30      100      100     700  -265  13060     1662     1   460   
       2021-10-01       50       50     600  -875   6640     1442     0   490   
       2021-10-02        0        0     100  -500   1460     1361     1   250   
       2021-10-03       50       50       0  -222   2500     1434     2   230   
       2021-10-04        0        0       0  -111   2420     1364     5   260   

type                후원  
       when             
amount 2021-09-13    0  
       2021-09-14    0  
       2021-09-15 -500  
       2021-09-16 -600  
       2021-09-17 -300  
       2021-09-18    0  
       2021-09-19    0  
       2021-09-20    0  
       2021-09-21    0  
       2021-09-22    0  
       2021-09-23    0  
       2021-09-24    0  
       2021-09-25 -100  
       2021-09-26    0  
       2021-09-27    0  
       2021-09-28 -100  
       2021-09-29 -100  
       2021-09-30    0  
       2021-10-01    0  
       2021-10-02    0  
       2021-10-03    0  
       2021-10-04    0

# 가입자수 추이

In [142]:
qrs = m.User.objects.filter(date_joined__gte='2021-09-13').values('date_joined__date').annotate(count=Count('date_joined__date'))

In [143]:
pd.DataFrame(qrs).set_index('date_joined__date').sort_index(ascending=True).iloc[:-1]

,count
date_joined__date,
2021-09-13,9
2021-09-14,3
2021-09-15,7
2021-09-16,14
2021-09-17,13
2021-09-18,10
2021-09-19,7
2021-09-20,6
2021-09-21,12


# 서베이별 참여자수

In [27]:
_boos = m.Boo.objects.filter(answers__has_key='12')

In [32]:
_boos[10].answers

{'7': {'47': [0],
  '48': [1],
  '49': [1],
  '50': [0],
  '51': [0],
  '172': [0],
  '173': [1],
  '174': [0],
  '176': [1],
  '177': [0],
  '178': [0],
  '180': [1],
  '181': [1],
  '182': [1],
  '183': [0],
  'finished': True},
 '12': {'80': [0],
  '81': [1],
  '82': [0],
  '83': [1],
  '84': [1],
  '85': ['티'],
  '86': ['ㅇ'],
  '87': [1],
  '88': [1],
  '89': [1],
  'finished': True},
 '19': {'238': [1], '239': [0], '240': [0], 'finished': True}}

In [ ]:
배드보이 4
터치그라운드 5
존버 6
포스토 7
브라스보트 8

비오는날 취향분석 9
슬기로운 캠필생활 10
나에게 딱 맞는 주말 베스트 코디 12
내 휴가 스타일 14
핫서머에 입기 좋은 핫팬츠 취향조사 15
구두취향 본격탐구 17
내 백 취향 18
요즘 막 당신의 관심사 19
비오는날 샌들 취향조사 20
어느 브랜드 좋아해 22
올가을 트랜드 조사 신발편 23
가을 유니폼 후디 맨투맨 고르기 24
부츠 이렇게 신어야지 25
이번 fw 뭐 신지 26


In [33]:
m.Boo.objects.filter(answers__has_key='4').count()

67

In [47]:
pd.DataFrame({
    '배드보이': [m.Boo.objects.filter(answers__has_key='4').count()],
    '터치그라운드': [m.Boo.objects.filter(answers__has_key='5').count()],
    '존버': [m.Boo.objects.filter(answers__has_key='6').count()],
    '포스토': [m.Boo.objects.filter(answers__has_key='7').count()],
    '브라스보트': [m.Boo.objects.filter(answers__has_key='8').count()],
    
    '비오는날 취향분석': [m.Boo.objects.filter(answers__has_key='9').count()],
    '슬기로운 캠핑생활': [m.Boo.objects.filter(answers__has_key='10').count()],
    '주말 베스트 코디': [m.Boo.objects.filter(answers__has_key='12').count()],
    '내 휴가 스타일': [m.Boo.objects.filter(answers__has_key='14').count()],
    '핫팬츠 취향조사': [m.Boo.objects.filter(answers__has_key='15').count()],
    '구두취향 본격탐구': [m.Boo.objects.filter(answers__has_key='17').count()],
    '내 백 취향': [m.Boo.objects.filter(answers__has_key='18').count()],
    '요즘 막 당신의 관심사': [m.Boo.objects.filter(answers__has_key='19').count()],
    '샌들 취향조사': [m.Boo.objects.filter(answers__has_key='20').count()],
    '어느 브랜드 좋아해': [m.Boo.objects.filter(answers__has_key='22').count()],
    '올가을 트랜드 조사 신발편': [m.Boo.objects.filter(answers__has_key='23').count()],
    '가을 유니폼 후디 맨투맨 고르기': [m.Boo.objects.filter(answers__has_key='24').count()],
    '부츠 이렇게 신어야지': [m.Boo.objects.filter(answers__has_key='25').count()]
}).T

,0
배드보이,67
터치그라운드,53
존버,47
포스토,67
브라스보트,53
비오는날 취향분석,53
슬기로운 캠핑생활,56
주말 베스트 코디,60
내 휴가 스타일,33
핫팬츠 취향조사,16


In [39]:
pd.DataFrame({'a':[1], 'b':[2]})

,a,b
0,1,2


# 포인트 현황

In [144]:
memb_list = ['quantlab@kakao.com',
'gem763@gmail.com',
'sejin76@nate.com',
'magrittelim@gmail.com',
'magrittelim@naver.com',
'smilekhm@gmail.com',
'hmkoo@kakao.com',
'giseok1929@daum.net',
'gentlebaek89@gmail.com',
'leesuyong@hotmail.com']

In [145]:
wallet_stat = []
for w in tqdm(m.Wallet.objects.all()[:]):
    if w.whose_type == 'boo':
        wallet_stat.append({
            'whose': str(w.whose), 
            'n_transaction': w.n_transaction, 
            'inflow': w.inflow, 
            'outflow': w.outflow, 
            'amount': w.amount
        })

  0%|          | 0/374 [00:00<?, ?it/s]

In [146]:
wallet_stat

[{'whose': 'quantlab | quantlab@kakao.com',
  'n_transaction': 688,
  'inflow': 53711,
  'outflow': 5485,
  'amount': 48226},
 {'whose': '모이버 | moiber@moiber.com',
  'n_transaction': 34825,
  'inflow': 0,
  'outflow': 537867,
  'amount': -537867},
 {'whose': '홈마 | quantlab@kakao.com',
  'n_transaction': 0,
  'inflow': 0,
  'outflow': 0,
  'amount': 0},
 {'whose': 'Seoul_domestic_m | magrittelim@gmail.com',
  'n_transaction': 1260,
  'inflow': 104546,
  'outflow': 23299,
  'amount': 81247},
 {'whose': 'minimal_ootd | magrittelim@gmail.com',
  'n_transaction': 0,
  'inflow': 0,
  'outflow': 0,
  'amount': 0},
 {'whose': 'giseok1929 | giseok1929@daum.net',
  'n_transaction': 248,
  'inflow': 53420,
  'outflow': 1538,
  'amount': 51882},
 {'whose': 'gentlebaek89 | gentlebaek89@gmail.com',
  'n_transaction': 1446,
  'inflow': 54540,
  'outflow': 0,
  'amount': 54540},
 {'whose': 'smilekhm | smilekhm@gmail.com',
  'n_transaction': 220,
  'inflow': 52292,
  'outflow': 4754,
  'amount': 47538}

In [147]:
df = pd.DataFrame(wallet_stat); df

,whose,n_transaction,inflow,outflow,amount
0,quantlab | quantlab@kakao.com,688,53711,5485,48226
1,모이버 | moiber@moiber.com,34825,0,537867,-537867
2,홈마 | quantlab@kakao.com,0,0,0,0
3,Seoul_domestic_m | magrittelim@gmail.com,1260,104546,23299,81247
4,minimal_ootd | magrittelim@gmail.com,0,0,0,0
...,...,...,...,...,...
330,minshin0907 | minshin0907@naver.com,151,1930,0,1930
331,hyundong.park0125 | hyundong.park0125@gmail.com,1,10,0,10
332,theloki7 | theloki7@naver.com,217,2000,0,2000
333,dldbwls12231 | dldbwls12231@naver.com,1,300,0,300


In [148]:
df.sort_values(by=['amount'], ascending=False).iloc[6:].iloc[:10]

,whose,n_transaction,inflow,outflow,amount
135,kojedong287 | kojedong287@naver.com,1952,4040,0,4040
172,skyehdgnl | skyehdgnl@naver.com,1850,3951,0,3951
143,zawon22 | zawon22@naver.com,1738,3620,0,3620
22,부캐 없~~다 | gap1212@naver.com,361,3014,111,2903
209,dydwk231 | dydwk231@naver.com,1242,3303,464,2839
133,kelifrona | kelifrona@naver.com,933,2800,0,2800
247,01062587408 | 01062587408@naver.com,1530,2780,0,2780
276,hsk1230 | hsk1230@hanmail.net,752,2770,0,2770
169,zaxa1019 | zaxa1019@naver.com,769,2906,177,2729
20,슈프림걸 | angelica.yum@gmail.com,441,2501,0,2501


# 래플참여 현황

In [53]:
raffle_result = trans.filter(type=101).values('sender__boo__user__email', 'receiver__raffle__item__name', 'amount')

In [73]:
df = pd.DataFrame(raffle_result)
df.columns = ['유저','래플','참여']
df = df[~df['유저'].isin(memb_list)]

In [74]:
df.groupby(by=['래플','유저']).count()

참여
래플                                  유저                       
KINTO TUMBLER 350ML                 dydwk231@naver.com      1
                                    fredasd0120@gmail.com   1
                                    heetae002@naver.com     7
                                    jueun0618@gmail.com     1
                                    jylee118218@naver.com   1
...                                                        ..
TAW&TOE ZEROVITY™ Flip Flop, SIZE M noahpark202@gmail.com   1
                                    rnstjr79@hanmail.net    2
                                    se05021@naver.com       1
                                    ssopie@gmail.com        1
                                    xogh8939@kakao.com      1

[64 rows x 1 columns]

In [75]:
df.groupby(by=['래플','유저']).count().to_excel('raffle_stat.xlsx')